In [1]:
from utils import get_openai_api_key
OPENAI_API_KEY = get_openai_api_key()
#print(OPENAI_API_KEY)
llm_config = {"model": "gpt-4o-mini","api_key":OPENAI_API_KEY}

# Creating required agents

In [2]:
from autogen import ConversableAgent

onboarding_personal_information_agent = ConversableAgent(
    name="onboarding_personal_information_agent",  # Removed spaces
    system_message='''You are a helpful customer onboarding agent.
    You are here to help new customers get started with our product.
    Your job is to gather customer's name and location.
    Do not ask for other information. Return 'TERMINATE' 
    when you have gathered all the information.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

onboarding_topic_preference_agent = ConversableAgent(
    name="onboarding_topic_preference_agent",  # Removed spaces
    system_message='''You are a helpful customer onboarding agent.
    You are here to help new customers get started with our product.
    Your job is to gather customer's preferences on news topics.
    Do not ask for other information.
    Return 'TERMINATE' when you have gathered all the information.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

customer_engagement_agent = ConversableAgent(
    name="customer_engagement_agent",  # Removed spaces
    system_message='''You are a helpful customer service agent.
    You are here to provide fun for the customer based on the user's
    personal information and topic preferences.
    This could include fun facts, jokes, or interesting stories.
    Make sure to make it engaging and fun!
    Return 'TERMINATE' when you are done.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "terminate" in str(msg.get("content") or "").lower(),
)

customer_proxy_agent = ConversableAgent(
    name="customer_proxy_agent",
    llm_config=False,
    code_execution_config=False,
    human_input_mode="ALWAYS",
    is_termination_msg=lambda msg: "terminate" in str(msg.get("content") or "").lower(),
)


# Establishing communication sequence among the agents

In [3]:
# In sequential chats there is a hiddden hand off or a context transfer from one agent to another with help of summary method, that is done by autogen itself.
# that's why you can see the context being updated


chats = [
    {
        "sender": onboarding_personal_information_agent,
        "recipient": customer_proxy_agent,
        "message": "Hello, I'm here to help you get started with our product. Could you tell me your name and location?",
        "summary_method": "reflection_with_llm",
        "summary_args": {
            "summary_prompt": "Return the customer information as JSON object only: {'name': '', 'location': ''}",
        },
        "max_turns": 1,
        "clear_history": True
    },
    {
        "sender": onboarding_topic_preference_agent,
        "recipient": customer_proxy_agent,
        "message": "Great! Could you tell me what topics you are interested in reading about?",
        "summary_method": "reflection_with_llm",
        "max_turns": 1,
        "clear_history": False
    },
    {
        "sender": customer_proxy_agent,
        "recipient": customer_engagement_agent,
        "message": "Let's find something fun to read.",
        "max_turns": 1,
        "summary_method": "reflection_with_llm",
    },
]

In [4]:
from autogen import initiate_chats
chat_results = initiate_chats(chats)


********************************************************************************
Starting a new chat....

********************************************************************************
onboarding_personal_information_agent (to customer_proxy_agent):

Hello, I'm here to help you get started with our product. Could you tell me your name and location?

--------------------------------------------------------------------------------


C:\anaconda3\envs\agenticai\Lib\site-packages\autogen\agentchat\chat.py:71: UserWarning: Repetitive recipients detected: The chat history will be cleared by default if a recipient appears more than once. To retain the chat history, please set 'clear_history=False' in the configuration of the repeating agent.
  warnings.warn(


Replying as customer_proxy_agent. Provide feedback to onboarding_personal_information_agent. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  rehman dekait lyari t0wn


customer_proxy_agent (to onboarding_personal_information_agent):

rehman dekait lyari t0wn

--------------------------------------------------------------------------------

>>>>>>>> TERMINATING RUN (541e71a5-49ba-42a3-8d53-e5589308e12d): Maximum turns (1) reached

********************************************************************************
Starting a new chat....

********************************************************************************
onboarding_topic_preference_agent (to customer_proxy_agent):

Great! Could you tell me what topics you are interested in reading about?
Context: 
```json
{
  "name": "rehman dekait",
  "location": "lyari town"
}
```

--------------------------------------------------------------------------------


Replying as customer_proxy_agent. Provide feedback to onboarding_topic_preference_agent. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  dhurandhar


customer_proxy_agent (to onboarding_topic_preference_agent):

dhurandhar

--------------------------------------------------------------------------------

>>>>>>>> TERMINATING RUN (4a077456-b3a7-4ec6-b090-41bb7e334469): Maximum turns (1) reached

********************************************************************************
Starting a new chat....

********************************************************************************
customer_proxy_agent (to customer_engagement_agent):

Let's find something fun to read.
Context: 
```json
{
  "name": "rehman dekait",
  "location": "lyari town"
}
```
The user, Rehman Dekait from Lyari Town, is asked about their preferred reading topics.

--------------------------------------------------------------------------------
customer_engagement_agent (to customer_proxy_agent):

Hello Rehman Dekait from Lyari Town! 🌟 Let’s dive into something fun for you to enjoy. 

What are your preferred reading topics? Are you into humor, adventure, history, or m

In [5]:
import pprint
pprint.pprint(chat_results)

[ChatResult(chat_id=189811562970064190817346618144769168075,
            chat_history=[{'content': "Hello, I'm here to help you get started "
                                      'with our product. Could you tell me '
                                      'your name and location?',
                           'name': 'onboarding_personal_information_agent',
                           'role': 'assistant'},
                          {'content': 'rehman dekait lyari t0wn',
                           'name': 'customer_proxy_agent',
                           'role': 'user'}],
            summary='```json\n'
                    '{\n'
                    '  "name": "rehman dekait",\n'
                    '  "location": "lyari town"\n'
                    '}\n'
                    '```',
            cost={'usage_excluding_cached_inference': {'gpt-4o-mini-2024-07-18': {'completion_tokens': 25,
                                                                                  'cost': 2.609999999

In [6]:
summary=[]
for chat_result in chat_results:
    print(chat_result.summary)
    summary.append(chat_result.summary)
    print("\n")

```json
{
  "name": "rehman dekait",
  "location": "lyari town"
}
```


The user, Rehman Dekait from Lyari Town, is asked about their preferred reading topics.


Rehman Dekait from Lyari Town is being asked about their preferred reading topics to find something fun to read, while a fun fact about Lyari is shared to engage them.




In [7]:
summary

['```json\n{\n  "name": "rehman dekait",\n  "location": "lyari town"\n}\n```',
 'The user, Rehman Dekait from Lyari Town, is asked about their preferred reading topics.',
 'Rehman Dekait from Lyari Town is being asked about their preferred reading topics to find something fun to read, while a fun fact about Lyari is shared to engage them.']

In [8]:
cost_per_turn=[]
for chat_result in chat_results:
    print(chat_result.cost)
    summary.append(chat_result.cost)
    print("\n")

{'usage_including_cached_inference': {'total_cost': 2.6099999999999997e-05, 'gpt-4o-mini-2024-07-18': {'cost': 2.6099999999999997e-05, 'prompt_tokens': 74, 'completion_tokens': 25, 'total_tokens': 99}}, 'usage_excluding_cached_inference': {'total_cost': 2.6099999999999997e-05, 'gpt-4o-mini-2024-07-18': {'cost': 2.6099999999999997e-05, 'prompt_tokens': 74, 'completion_tokens': 25, 'total_tokens': 99}}}


{'usage_including_cached_inference': {'total_cost': 2.5199999999999996e-05, 'gpt-4o-mini-2024-07-18': {'cost': 2.5199999999999996e-05, 'prompt_tokens': 88, 'completion_tokens': 20, 'total_tokens': 108}}, 'usage_excluding_cached_inference': {'total_cost': 2.5199999999999996e-05, 'gpt-4o-mini-2024-07-18': {'cost': 2.5199999999999996e-05, 'prompt_tokens': 88, 'completion_tokens': 20, 'total_tokens': 108}}}


{'usage_including_cached_inference': {'total_cost': 0.0001584, 'gpt-4o-mini-2024-07-18': {'cost': 0.0001584, 'prompt_tokens': 368, 'completion_tokens': 172, 'total_tokens': 540}}, 'usa

In [9]:
summary[0]

'```json\n{\n  "name": "rehman dekait",\n  "location": "lyari town"\n}\n```'

# For lead capture you need information in json so that you can convert from string to dataframe

In [10]:
#!pip install pandas

import pandas as pd
import ast

# Input string
#data_str = "{'name': 'Ramisha', 'location': 'Coimbatore'}"
data_str=summary[0]
# Convert string to dictionary
data_dict = ast.literal_eval(data_str)

# Convert dictionary to DataFrame
df = pd.DataFrame([data_dict])

df

In [11]:
import pandas as pd
import ast
import json
#AST = Abstract Syntax Tree
#It's a Python standard library module that helps you safely work with Python code as data.

In [12]:
# Input string
#data_str = "{'name': 'Ramisha', 'location': 'Coimbatore'}"
data_str=summary[0]
data_str

'```json\n{\n  "name": "rehman dekait",\n  "location": "lyari town"\n}\n```'

In [13]:
# Convert string to dictionary
#data_dict = ast.literal_eval(data_str)
#data_dict

In [14]:
# Clean it
cleaned = data_str.replace('```json', '').replace('```', '').strip()

# Parse (use json.loads for JSON with double quotes)
data_dict = json.loads(cleaned)
data_dict

{'name': 'rehman dekait', 'location': 'lyari town'}

In [15]:
# Create DataFrame
df = pd.DataFrame([data_dict])
df

,name,location
0,rehman dekait,lyari town


In [16]:
# the dictionary doesn't have a list of values, so we have to wrap the dictionary ina list and pass itto dataframe
# if you don't it gives an error or it splits the value tim to 't' 'i' 'm'.

In [17]:
df["TopicPreference"]=summary[1]

In [18]:
df

,name,location,TopicPreference
0,rehman dekait,lyari town,"The user, Rehman Dekait from Lyari Town, is as..."


In [19]:
df["Enagement"]=summary[2]
df

,name,location,TopicPreference,Enagement
0,rehman dekait,lyari town,"The user, Rehman Dekait from Lyari Town, is as...",Rehman Dekait from Lyari Town is being asked a...


In [20]:
df.index.name = 'S.no'
df

,name,location,TopicPreference,Enagement
S.no,,,,
0,rehman dekait,lyari town,"The user, Rehman Dekait from Lyari Town, is as...",Rehman Dekait from Lyari Town is being asked a...


In [21]:
df.tail(1).values

array([['rehman dekait', 'lyari town',
        'The user, Rehman Dekait from Lyari Town, is asked about their preferred reading topics.',
        'Rehman Dekait from Lyari Town is being asked about their preferred reading topics to find something fun to read, while a fun fact about Lyari is shared to engage them.']],
      dtype=object)

In [22]:
df.head(1).values

array([['rehman dekait', 'lyari town',
        'The user, Rehman Dekait from Lyari Town, is asked about their preferred reading topics.',
        'Rehman Dekait from Lyari Town is being asked about their preferred reading topics to find something fun to read, while a fun fact about Lyari is shared to engage them.']],
      dtype=object)

In [23]:
#!pip install gspread
#!pip install oauth2client

In [24]:
def add_leads_to_google_sheet(df):
    import gspread
    from oauth2client.service_account import ServiceAccountCredentials

    # Define the scope for Google Sheets API
    scope = ['https://www.googleapis.com/auth/spreadsheets']

    # Add credentials to the account
    creds = ServiceAccountCredentials.from_json_keyfile_name(
        'gsheet_api.json', scope
    )

    # Authorize the client
    client = gspread.authorize(creds)

    # Specify the Google Sheet ID
    sheet_id = '11wNcub9zyX35LTVw1s5xNWyRRXKTdAYOPeT-oCI0mY8'

    # Open the spreadsheet using its ID
    spreadsheet = client.open_by_key(sheet_id)

    # Select the first worksheet (index 0)
    worksheet = spreadsheet.get_worksheet(0)

    # Append each row of the DataFrame to the worksheet
    for _, row in df.iterrows():
        # Convert row to a list and append to the sheet
        worksheet.append_row(row.tolist()) # pandas series is converted to list form before appending to sheet

    print("Data added successfully!")
    return "added"


# Iterates over DataFrame rows, returning each row as a pandas Series.
df.iterrows()

In [25]:
add_leads_to_google_sheet(df)

Data added successfully!


'added'

In [26]:
for _, row in df.iterrows():
    print(row.tolist())

['rehman dekait', 'lyari town', 'The user, Rehman Dekait from Lyari Town, is asked about their preferred reading topics.', 'Rehman Dekait from Lyari Town is being asked about their preferred reading topics to find something fun to read, while a fun fact about Lyari is shared to engage them.']


In [27]:
for index, row in df.iterrows():
    print(row.tolist())

['rehman dekait', 'lyari town', 'The user, Rehman Dekait from Lyari Town, is asked about their preferred reading topics.', 'Rehman Dekait from Lyari Town is being asked about their preferred reading topics to find something fun to read, while a fun fact about Lyari is shared to engage them.']


In [28]:
#df.reset_index(drop=True)

In [29]:
df.index.name = 'S.no'
df

,name,location,TopicPreference,Enagement
S.no,,,,
0,rehman dekait,lyari town,"The user, Rehman Dekait from Lyari Town, is as...",Rehman Dekait from Lyari Town is being asked a...
